<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/notebooks/Section_E_Manual_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Import packages

In [ ]:
from google.colab import drive
import ee
import geemap
from google.colab import drive
import numpy as np
import time
import pandas as pd
from google.colab import drive

# MODIFIABLE VARIABLE ALERT

In [ ]:
# Enter in the Google Earth Engine Folder in which your flood maps and reference data are located. Make sure the string ends in a slash
my_gee_folder = "users/mickymags/oct_cambodia/"

# Enter in the Google Earth Engine Project you will use
my_gee_project = 'servir-sco-assets'

# Enter in the Google Drive Folder you wish to use. Make sure this ends in a slash, This should start with "drive/MyDrive/..."
my_gdrive_folder = "drive/MyDrive/Flood_Intercomparison/Case_Studies/oct/oct_cambodia"

# Enter in the date of interest in YYYY-MM-dd format
doi = "2024-10-01"

# Enter in a description of the event. This can be whatever you want it to be.
event_desc = 'oct_cambodia'

In [ ]:
ee.Authenticate()

ee.Initialize(project=my_gee_project)

In [ ]:
# Define the path to the area of interest and extract the latitude and longitude for mapping purposes.
aoi = ee.FeatureCollection(my_gee_folder + "aoi")
roi = aoi.geometry()
aoi_centroid = aoi.geometry().centroid()             # Get the center of the AOI
lon = aoi_centroid.coordinates().get(0).getInfo()    # Extract the longitude from the centroid
lat = aoi_centroid.coordinates().get(1).getInfo()    # Extract the latitude from the centroid

# Step 2: Validating Module 6

Import each of the flood maps we exported at the end of Module 3, and the reference data we collected in Collect Earth Online.  

In [ ]:
# Define the paths to the flood maps and reference data
my_ref_data = ee.FeatureCollection(my_gee_folder + "reference_data")
dswx_hls = ee.Image(my_gee_folder + "dswxhls_harmonized")
dswx_s1 = ee.Image(my_gee_folder + "dswxs1_harmonized")

In [ ]:
# Define the mask of each flood map
dswxhls_mask = dswx_hls.neq(2)
dswxs1_mask = dswx_s1.neq(2)

In [ ]:
# Update all of the masks for each of the flood maps
dswxhls_final = dswx_hls.updateMask(dswxhls_mask)
dswxs1_final = dswx_s1.updateMask(dswxs1_mask)

In [ ]:
vp = {
    'min': 0,
    'max': 1,
    'palette': ['FF0000', 'add8e6'],
    'alpha': 0.3
}

Module 6 was validated by changing the "i" counter below for each integer between 0 and 2704 (the length of the reference data array), then inspecting the map classification versus the reference classification at each point

In [ ]:
i = 0

ref_size = my_ref_data.size().getInfo()
ref_list = my_ref_data.toList(ref_size)
ref1 = ref_list.get(i)

ref1_dic = ref1.getInfo()
ref_coords = ref1_dic['geometry']['coordinates']
ref_lon = ref_coords[0]
ref_lat = ref_coords[1]

cloud_percent = ref1_dic["properties"]['water?:cloud']
water_percent = ref1_dic["properties"]['water?:yes']
nonwater_percent = ref1_dic["properties"]['water?:no']
plot_id = ref1_dic["properties"]["\ufeffplotid"]

print('Plotid:', plot_id)
print("Confidence:",ref1_dic["properties"]["confidence"])
print('\ncloud percent:', cloud_percent)
print('water percent:', water_percent)


Map = geemap.Map(center = (ref_lat, ref_lon), zoom = 19)
layers = Map.layers
Map.addLayer(dswx_hls, vp, 'DSWx-HLS')
Map.addLayer(my_ref_data, {}, 'Reference Data')
Map

# Step 3: Validating Module 9

In [ ]:
iou_geom = ee.Geometry.Rectangle([105.175576697908, 11.0296195510959, 105.1783281076038, 11.05670676485824])

In [ ]:
dswxhls_sample = dswx_hls.sample(
    region = iou_geom,
    scale = 30,
    projection = 'EPSG:32648',
    geometries = True
)

dswxs1_sample = dswx_s1.sample(
    region = iou_geom,
    scale = 30,
    projection = 'EPSG:32648',
    geometries = True
)

In [ ]:
alt_vp = {
    'min': 0,
    'max': 1,
    'palette': ['00FF00', '0000FF'],
    'alpha': 0.3
}

# Manual IoU calculation

In [ ]:
i = 970

dswxhls_sample_listed = dswxhls_sample.toList(1000)
dswxhls_poi = ee.Feature(dswxhls_sample_listed.get(i)).geometry().getInfo()
dswxhls_coordinates = dswxhls_poi['coordinates']
dswxhls_lon = dswxhls_coordinates[0]
dswxhls_lat = dswxhls_coordinates[1]

point = ee.Geometry.Point(dswxhls_lon, dswxhls_lat)

print(i)

Map = geemap.Map(center = (dswxhls_lat, dswxhls_lon), zoom = 18)
layers = Map.layers
Map.addLayer(dswx_hls, vp, 'DSWx-HLS')
#Map.addLayer(dswx_s1, alt_vp, 'DSWx-S1')
Map.addLayer(iou_geom)
Map.addLayer(point, {}, 'Point of Interest')
Map


In [ ]:
def iou(img1, img2, desc1, desc2, aoi, myproj, mydesc):
  img1_renamed = img1.rename(desc1)
  img2_renamed = img2.rename(desc2)
  #id = flood_event_desc + '_' + desc1 + '_' + desc2

  # Combine img1_renamed annd img2_renamed into a single image containing the information of each in a separate band.
  combo = img1_renamed.addBands(img2_renamed)

  sample = combo.sample(
    region= aoi,
    scale=30,
    projection = myproj,
    #numPixels = 1e13#num_pixels
    )

  geemap.ee_export_vector_to_drive(
      collection=sample,
      description= 'validating_module_9_v5',
      fileFormat='CSV',
      folder='valmod9',
      selectors = [desc1, desc2]
    )

  time.sleep(120)

  #Create an empty dataframe
  dataframes = []
  combined_dataframe = pd.DataFrame()

  data = pd.read_csv('validating_module_9_v5.csv')
  combined_dataframe = pd.concat([combined_dataframe, data])

  ######### Calculate Intersection Over Union ##########
  intersection = 0
  union = 0

  # Let the user know where we are at in the workflow
  print('calculating iou...')

  # Get the number of rows
  num_rows = int(np.floor(combined_dataframe.size / 2))

  # For each row
  for j in range(num_rows):

    # Get the row of interest (this will represnt a single pixel in img1 and img2)
    row = combined_dataframe.iloc[j]

    # Get the cell corresponding to the pixel information from img1
    feat1 = row[desc1]

    # Get the cell corresponding to the pixel information from img2
    feat2 = row[desc2]

    # Percent of rows that have been completed
    perc = j * 100 / num_rows

    # Every so often, inform the user how close we are to completing this part of the code
    if j % 50 == 0:
      print('iou calculation is {0:0.1f} % complete'.format(perc))

    # If both features are masked, ignore
    if feat1 == 2 or feat2 == 2:
      continue

    # if both features are one, add one to the intersection
    if feat1 == 1 and feat2 == 1:
      intersection += 1

    # if either feature is one, add one to the union count
    if feat1 == 1 or feat2 == 1:
      union += 1

  return [id, intersection/union]

In [ ]:
drive.mount('/content/drive/')

In [ ]:
pwd

In [ ]:
ls

In [ ]:
cd ..

In [ ]:
cd valmod9

In [ ]:
cd drive/My Drive/Flood_Intercomparison

In [ ]:
ls

In [ ]:
iou(dswx_s1, dswx_hls, 'dswx_s1', 'dswx_hls', iou_geom, 'EPSG:32648', 'mod9val')